In [ ]:
import anndata
import scvelo as scv
import pandas as pd
import numpy as np
import matplotlib as plt
%load_ext rpy2.ipython

In [ ]:
#matplotlib visualization settings
scv.set_figure_params('scvelo')
scv.settings.presenter_view = True 

In [ ]:
#Load loom files
gfp_loom = anndata.read_loom("data/3575-LT.loom")
tdtom_loom = anndata.read_loom("data/5291-LT.loom")

In [ ]:
gfp_loom.var_names_make_unique()
tdtom_loom.var_names_make_unique()

In [ ]:
#Load cellID extracted from Seurat object
endothelium_obs = pd.read_csv("../sox17_seurat4_remap/output/for_scvelo/Endothelium_cellID_obs.csv")

In [ ]:
#Process cellID
endothelium_obs_gfp = endothelium_obs[endothelium_obs['x'].str.contains('gfp_')]
endothelium_obs_tdtom = endothelium_obs[endothelium_obs['x'].str.contains('tdtomato_')]
gfp_loom.obs.index = gfp_loom.obs.index.str.replace("3575-LT:","gfp_")
gfp_loom.obs.index = gfp_loom.obs.index.str.replace("x","-1")
tdtom_loom.obs.index = tdtom_loom.obs.index.str.replace("5291-LT:","tdtomato_")
tdtom_loom.obs.index = tdtom_loom.obs.index.str.replace("x","-1")

In [ ]:
#Filter loom files for processed cellIDs
gfp_loom = gfp_loom[np.isin(gfp_loom.obs.index, endothelium_obs_gfp["x"])]
tdtom_loom = tdtom_loom[np.isin(tdtom_loom.obs.index, endothelium_obs_tdtom["x"])]

In [ ]:
#Merge gfp and tdtomato looms
endothelium_loom = gfp_loom.concatenate(tdtom_loom)

In [ ]:
#Clean up index
endothelium_loom.obs.index = endothelium_loom.obs.index.str.replace("-0","")
endothelium_loom.obs.index = endothelium_loom.obs.index.str.replace("-1-1","-1")

In [ ]:
#Load UMAP coordinates of Seurat object
endothelium_umap_cord = pd.read_csv("../sox17_seurat4_remap/output/for_scvelo/Endothelium_cell_embeddings.csv")

In [ ]:
#Reorder coordinates
endothelium_loom_index = pd.DataFrame(endothelium_loom.obs.index)
endothelium_loom_index = endothelium_loom_index.rename(columns = {0:'Cell ID'})
endothelium_loom_index

endothelium_umap_cord = endothelium_umap_cord.rename(columns = {'Unnamed: 0':'Cell ID'})
endothelium_umap_cord

endothelium_umap_cord_ordered = endothelium_loom_index.merge(endothelium_umap_cord, on = "Cell ID")

endothelium_umap_cord_ordered = endothelium_umap_cord_ordered.iloc[:,1:]
endothelium_loom.obsm['X_umap'] = endothelium_umap_cord_ordered.values

In [ ]:
#Clusters and their cluster colors can be added in the same fashion 
endothelium_clusters = pd.read_csv("../sox17_seurat4_remap/output/for_scvelo/Endothelium_clusters.csv")
endothelium_clusters = endothelium_clusters.rename(columns = {'Unnamed: 0':'Cell ID'})
endothelium_clusters_ordered = endothelium_loom_index.merge(endothelium_clusters, on = "Cell ID")
endothelium_clusters_ordered = endothelium_clusters_ordered.iloc[:,1:]

#Instead of adding them as an multidimensional observation ('obsm'), we'd add them under the unstructured annotation 'uns.'
endothelium_loom.obs['Clusters'] = endothelium_clusters_ordered.values

In [ ]:
#Proportions of spliced/unspliced counts 
scv.pl.proportions(endothelium_loom)

In [ ]:
#scVelo run
scv.pp.filter_and_normalize(endothelium_loom)
scv.pp.moments(endothelium_loom)
scv.tl.recover_dynamics(endothelium_loom)
scv.tl.velocity(endothelium_loom, mode = "dynamical")
scv.tl.velocity_graph(endothelium_loom)
scv.pl.velocity_embedding(endothelium_loom, basis = 'umap')
scv.pl.velocity_embedding_stream(endothelium_loom, basis = 'umap', color = endothelium_loom.uns['Cluster_colors'], colorbar = False)

In [ ]:
#Add pseudotime values from Monocle3 analysis 
endothelium_pseudotime = pd.read_csv("../sox17_seurat4_remap/output/for_scvelo/Endothelium_pseudotime.csv")
endothelium_pseudotime = endothelium_pseudotime.rename(columns = {'Unnamed: 0':'Cell ID'})
endothelium_pseudotime_ordered = endothelium_loom_index.merge(endothelium_pseudotime, on = "Cell ID")
endothelium_pseudotime_ordered = endothelium_pseudotime_ordered.iloc[:,1:]
endothelium_loom.uns['Pseudotime'] = endothelium_pseudotime_ordered.values

In [ ]:
#Figure_3C
scv.pl.velocity_embedding_stream(endothelium_loom, basis = 'umap', color = endothelium_loom.uns['Pseudotime'], cmap='plasma')